In [1]:
%load_ext autoreload
%autoreload 2
from course_helpers import *
from helpers import *


In [ ]:
#pickle_vocab(vocab_cut, vocab_pkl)
#cooc_(vocab_pkl, DATA_PATH, coco_pkl)
#glove(coco_pkl, embd)

In [2]:
with open(coco_pkl, "rb") as f:
        cooc = pickle.load(f)
print(cooc.shape)
with open(vocab_pkl, "rb") as f:
        vocab = pickle.load(f)
print(len(vocab))
embedding = np.load(embd+'.npy')
print(embedding.shape)

(21161, 21161)
21161
(21161, 20)


In [44]:
train_pos, train_neg, test_set = read_data(DATA_PATH)
train_set = pd.concat([train_pos, train_neg],  axis=0)
glove_embd = embedding

In [4]:
train_set.loc[:, train_set.columns!='sentiment'] = train_set.applymap(lambda x: find_token(x, vocab))
test_set2 =  test_set.applymap(lambda x: find_token(x, vocab))
#train_set.loc[:, train_set.columns!='sentiment'] = train_set.loc[:, train_set.columns!='sentiment'].applymap(lambda x: findembedding(x, glove_embd))

C:\Users\hysam\AppData\Local\Temp\ipykernel_17652\3939322011.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  train_set.loc[:, train_set.columns!='sentiment'] = train_set.applymap(lambda x: find_token(x, vocab))


In [5]:
# Deep learning: 
import tensorflow as tf
from keras.models import Model
from keras.layers import LSTM, Dense, Embedding, concatenate, Dropout, concatenate, Input
from keras.layers import Bidirectional

class RnnModel():
    """
    A recurrent neural network for semantic analysis
    """
    def __init__(self, embedding_matrix, embedding_dim, max_len, X_additional=None):
        
        inp1 = Input(shape=(max_len,))
        x = Embedding(embedding_matrix.shape[0], embedding_dim, weights=[embedding_matrix])(inp1)
        x = Bidirectional(LSTM(256, return_sequences=True))(x)
        x = Bidirectional(LSTM(150))(x)
        x = Dense(128, activation="relu")(x)
        x = Dropout(0.1)(x)
        x = Dense(64, activation="relu")(x)
        x = Dense(1, activation="sigmoid")(x)    
        model = Model(inputs=inp1, outputs=x)

        model.compile(loss = 'binary_crossentropy', optimizer = 'adam')
        self.model = model

In [20]:
# Creating the model
embed_dim = glove_embd.shape[1]
max_len = train_set.shape[1]-1
X_train = train_set.loc[:, train_set.columns!='sentiment']
Y_train = train_set['sentiment']

batch_size = 16
epochs = 5

In [9]:
X_subset =  [ X_train[:2000],  X_train[-2000:]]
X_subset = pd.concat(X_subset)
Y_subset =  [ Y_train[:2000],  Y_train[-2000:]]
Y_subset = pd.concat(Y_subset)

C:\Users\hysam\AppData\Local\Temp\ipykernel_17652\1328371404.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  Y_subset =  [ Y_train[:2000],  Y_train[-2000:]]


In [21]:
rnn = RnnModel(
    embedding_matrix=glove_embd, 
    embedding_dim=embed_dim, 
    max_len=max_len
)
rnn.model.fit(
    X_train,
    Y_train, 
    batch_size=batch_size, 
    epochs=epochs
)

model = rnn.model


Epoch 1/5
12500/12500 [==============================] - 3342s 267ms/step - loss: 0.4563
Epoch 2/5
12500/12500 [==============================] - 6997s 560ms/step - loss: 0.3581
Epoch 3/5
12500/12500 [==============================] - 4668s 373ms/step - loss: 0.3304
Epoch 4/5
12500/12500 [==============================] - 4018s 321ms/step - loss: 0.3089
Epoch 5/5
12500/12500 [==============================] - 5483s 439ms/step - loss: 0.2844


In [22]:
preds1 = rnn.model.predict(X_train[-2500:-2100])
preds2=rnn.model.predict(X_train[2100:2500])
preds = np.append(preds1, preds2).tolist()
gndt = np.append(Y_train[-2500:-2100], Y_train[2100:2500]).tolist()

13/13 [==============================] - 1s 51ms/step


C:\Users\hysam\AppData\Local\Temp\ipykernel_17652\1386457197.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  gndt = np.append(Y_train[-2500:-2100], Y_train[2100:2500]).tolist()


In [23]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(gndt, [1 if x > 0.5 else 0 for x in preds])
acc

0.89625

#predicting on the test set

In [47]:
preds_test = rnn.model.predict(test_set2)
preds = np.ones_like(preds_test)
preds[np.where(preds_test<0.5)] = -1
preds = pd.DataFrame(preds)

313/313 [==============================] - 13s 43ms/step


In [63]:
preds

,0
0,-1.0
1,-1.0
2,-1.0
3,-1.0
4,-1.0
...,...
9995,1.0
9996,-1.0
9997,-1.0
9998,1.0


In [68]:
import csv
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in .csv format for submission to Kaggle or AIcrowd
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, "w") as csvfile:
        fieldnames = ["Id", "Prediction"]
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({"Id": int(r1)+1, "Prediction": int(r2)})

create_csv_submission(preds.index.values.tolist(), np.asarray(preds), "submission1_basic_lstm_bad_token.csv")

#Plotting for tokenization purposes

In [ ]:
#plotting the number of occurence of different words
import matplotlib.pyplot as plt
import pandas as pd
def read_file(filename):
    """
    DESCRIPTION: 
            Reads a file and returns it as a list
    INPUT: 
            filename: Name of the file to be read
    """
    data = []
    with open(filename, "r", encoding='utf8') as ins:
        for line in ins:
            #line = line.split(' ')
            data.append(line.strip())
    return data
full_vocab = pd.DataFrame(read_file('vocab_full.txt'))
full_vocab = full_vocab[0].str.split(' ', expand=True)
full_vocab = full_vocab.astype({0:'int'})
full_vocab.sort_values(by=[0], ascending=False, inplace= True)
x = full_vocab[1]
y = full_vocab[0]
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(x,y)
plt.show()